In [1]:
import pandas as pd

In [3]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f"{github_url}?raw=1"
df = pd.read_csv(url)
df = df.iloc[:300]
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [4]:
model_name = "multi-qa-mpnet-base-dot-v1"

from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name)

/home/alexey/miniconda3/envs/llm-zoomcamp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
answer_llm = df.iloc[0].answer_llm
print(embedding_model.encode(answer_llm)[0])

-0.4224467


In [7]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [8]:
from tqdm.notebook import tqdm

tqdm.pandas()

In [9]:
import numpy as np

In [10]:
evaluations = []

for row in tqdm(df.to_dict(orient="records")):
    v_llm = embedding_model.encode(row["answer_llm"])
    v_orig = embedding_model.encode(row["answer_orig"])
    dot_prod = np.array(v_llm).dot(np.array(v_orig))
    evaluations.append(dot_prod)

  0%|          | 0/300 [00:00<?, ?it/s]

In [12]:
pd.DataFrame(evaluations).describe()

,0
count,300.000000
mean,27.495996
std,6.384744
min,4.547922
25%,24.307846
50%,28.336862
75%,31.674306
max,39.476021


In [13]:
evaluations = []

for row in tqdm(df.to_dict(orient="records")):
    v_llm = embedding_model.encode(row["answer_llm"])
    norm = np.sqrt((v_llm * v_llm).sum())
    v_llm = v_llm / norm
    v_orig = embedding_model.encode(row["answer_orig"])
    norm = np.sqrt((v_orig * v_orig).sum())
    v_orig = v_orig / norm
    dot_prod = np.array(v_llm).dot(np.array(v_orig))
    evaluations.append(dot_prod)

pd.DataFrame(evaluations).describe()

  0%|          | 0/300 [00:00<?, ?it/s]

,0
count,300.000000
mean,0.728392
std,0.157755
min,0.125357
25%,0.651273
50%,0.763761
75%,0.836235
max,0.958796


In [14]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from rouge import Rouge

rouge_scorer = Rouge()

r = df.to_dict(orient="records")[10]
print(r)

scores = rouge_scorer.get_scores(r["answer_llm"], r["answer_orig"])[0]
print(scores)

{'answer_llm': "Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.", 'answer_orig': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.', 'document': '5170565b', 'question': 'Are sessions recorded if I miss one?', 'course': 'machine-learning-zoomcamp'}
{'rouge-1': {'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}, 'rouge-2': {'r': 0.21621621621621623, 'p': 0.21621621621621623, 'f': 0.21621621121621637}, 'rouge-l': {'r': 0.3939393939393939, 'p': 0.3939393939393939, 'f': 0.393939388939394}}


In [17]:
print(scores["rouge-1"]["f"])

0.45454544954545456


In [18]:
np.mean([scores["rouge-1"]["f"], scores["rouge-2"]["f"], scores["rouge-l"]["f"]])

0.35490034990035496

In [19]:
def get_rouge_2(llm, orig):
    return rouge_scorer.get_scores(llm, orig)[0]["rouge-2"]["f"]

In [21]:
df.columns

Index(['answer_llm', 'answer_orig', 'document', 'question', 'course'], dtype='object')

In [22]:
df["rouge-2"] = df.progress_apply(
    lambda row: get_rouge_2(row["answer_llm"], row["answer_orig"]), axis=1
)

  0%|          | 0/300 [00:00<?, ?it/s]

In [23]:
df["rouge-2"].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge-2, dtype: float64